In [1]:
import sys
sys.path.insert(1, './lib')
import time
import csv_io
import ontoinfo
import gitcrawl
from github import Github
from github import Auth
from pygit2 import Repository
import glob
import re

In [2]:
auth = Auth.Token("YOUR TOKEN")
git = Github(auth=auth)

In [3]:
user="OCDO"
folder="test2"

In [6]:
#OUTPUT PATHs
outname = "Output/GitTest.csv"
csv_io.create_csv(outname)

In [13]:
def get_ai_model():
    from openai import AzureOpenAI
    ai_client = AzureOpenAI(
      azure_endpoint = "https://my-east.openai.azure.com/", 
      api_key="YOUR API KEY",  
      api_version="2023-05-15"
    )

    #modelName="Microsoft.CognitiveServicesOpenAI-20240111194907"
    #deployment_name='gpt-35'
    return ai_client

try:
    my_ai = get_ai_model()
    start_phrase = "say hello"
    
    response = my_ai.chat.completions.create(
        model="gpt-35", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": start_phrase}
        ]
    )
    #print(response.choices[0].message.content.format(1))
    print("AI connected")
except Exception as e:
    print("An error occurred:", e)
    my_ai = []

An error occurred: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}


In [5]:
repo_list = gitcrawl.get_user_repos(user,git)

['.github', 'asmo', 'cdco', 'cdos', 'cmso-ontology', 'DSIM', 'ldo', 'mmss', 'pldo', 'podo']


In [14]:
for repo in repo_list:
        time.sleep(1)
        print("NEXT REPO")
        prefix="https://github.com"
        subfold="/"+user+"/"+repo
        print("link=" + prefix + subfold)
        repath = folder+subfold 
        try:
            clone_repo(folder,user,repo)
        except:
            print("repo exists?")
            pass
            
        try:
            repobj = Repository(repath)
        except:
             print("repo failed to found?")
             continue

        #INFO FROM REPOSITORY ITSELF
        inidate = gitcrawl.get_first_last_commit(repobj)[1]   #first commit date

        if (my_ai!=[]):
            read_info = gitcrawl.extract_readme(repath,my_ai)        #parse README.md if available (with AI currently)
        else:
            read_info = ["no info","no info","no info"]       #otherwise no info - contact point, related project, documentation link
    
        lic_info = gitcrawl.extract_license(repath)        #get first line of the LICENSE file
    
        #looking for onto files
        onto_infos=[]
        scores=[]
        maxscore=-1
        for ext in ontoinfo.ext_list:  #check all ontology extensions
                files = glob.glob(repath + '/**/*.' + ext, recursive=True)
                for fi in files:
                    success = ontoinfo.querry_successful(fi)  #test querryability of the file
                    if (success):

                        #LINK TO THE FILE (default branch)
                        x=fi[len(folder):].replace("\\","/")
                        ind=[x.start() for x in re.finditer('/',x)][2]
                        branch = Repository(folder + x[:ind]).head.shorthand
                        gitlink = x[:ind] + "/tree/" + branch + x[ind:]
                        #print(gitlink)
                        
                        # LINK TO REPOSITORY
                        #gitlink = prefix + subfold

                        #extract information from ontology file
                        oinfo_fill = ontoinfo.extract_info(fi,gitlink)   

                        #add to the csv file
                        csv_io.add_to_csv(outname,oinfo_fill[0],oinfo_fill[1],oinfo_fill[2],oinfo_fill[3],oinfo_fill[4],lic_info,read_info[1],read_info[0],read_info[2],inidate,"owl:Ontology")

NEXT REPO
link=https://github.com/OCDO/.github
repo exists?
21.9.2024
29.1.2024
NEXT REPO
link=https://github.com/OCDO/asmo
repo exists?
21.9.2024
2.2.2023



1,Atomistic Simulation Methods Ontology (ASMO),http://purls.helmholtz-metadaten.de/asmo/, does not look like a valid URI, trying to serialize this will break.

1,Atomistic Simulation Methods Ontology (ASMO),http://purls.helmholtz-metadaten.de/asmo/,https://github.com/OCDO/asmo/tree/main/asmo.owl, does not look like a valid URI, trying to serialize this will break.

1,Atomistic Simulation Methods Ontology (ASMO),http://purls.helmholtz-metadaten.de/asmo/,https://github.com/OCDO/asmo/tree/main/asmo.owl,https://orcid.org/0000-0001-7564-7990, does not look like a valid URI, trying to serialize this will break.

1,Atomistic Simulation Methods Ontology (ASMO),http://purls.helmholtz-metadaten.de/asmo/,https://github.com/OCDO/asmo/tree/main/asmo.owl,https://orcid.org/0000-0001-7564-7990,Attribution 4.0 International, does not look like a valid URI, trying to serialize this will break.

1,Atomistic Simulation Methods Ontology (ASMO),http://purls.helmholtz-metadaten.de/asmo/,https://github.com/O

http://purls.helmholtz-metadaten.de/asmo/
Atomistic Simulation Methods Ontology (ASMO)
https://orcid.org/0000-0001-7564-7990
https://github.com/OCDO/asmo/tree/main/asmo.owl



2,Crystallographic Defect Core Ontology,http://purls.helmholtz-metadaten.de/cdos/cdco, does not look like a valid URI, trying to serialize this will break.

2,Crystallographic Defect Core Ontology,http://purls.helmholtz-metadaten.de/cdos/cdco,https://github.com/OCDO/cdco/tree/main/cdco.owl, does not look like a valid URI, trying to serialize this will break.

2,Crystallographic Defect Core Ontology,http://purls.helmholtz-metadaten.de/cdos/cdco,https://github.com/OCDO/cdco/tree/main/cdco.owl,https://orcid.org/0000-0001-7564-7990, does not look like a valid URI, trying to serialize this will break.

2,Crystallographic Defect Core Ontology,http://purls.helmholtz-metadaten.de/cdos/cdco,https://github.com/OCDO/cdco/tree/main/cdco.owl,https://orcid.org/0000-0001-7564-7990,no info, does not look like a valid URI, trying to serialize this will break.

2,Crystallographic Defect Core Ontology,http://purls.helmholtz-metadaten.de/cdos/cdco,https://github.com/OCDO/cdco/tree/main/cdco.owl,https://o

NEXT REPO
link=https://github.com/OCDO/cdco
repo exists?
12.4.2024
2.2.2024
http://purls.helmholtz-metadaten.de/cdos/cdco
Crystallographic Defect Core Ontology
https://orcid.org/0000-0001-7564-7990
https://github.com/OCDO/cdco/tree/main/cdco.owl
NEXT REPO
link=https://github.com/OCDO/cdos
repo exists?
30.1.2024
30.1.2024


KeyboardInterrupt: 